# "Title"
> "Awesome summary"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [ ]:
!pip install -Uqq transformers datasets tqdm
!pip install -Uqq ohmeow-blurr
!pip install -Uqq wandb

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import wandb

from transformers import *
from fastai.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb_init_kwargs = {
    'reinit': True, 
    'project': "fake-news-blurr", 
    'entity': "tasmiah-tahsin",
    'notes': 'Finetuning banfakenews with multilingual bert via Blurr', 
    'tags': ['mbert', 'fake-news-classification', 'blurr']
  }

# Load dataset

In [ ]:
fake = pd.read_csv("/content/Fake-1K.csv")
authentic = pd.read_csv("/content/Authentic-48K.csv",engine='python',error_bad_lines=False,warn_bad_lines=True)
df = pd.concat([authentic[['headline','content','label']],fake[['headline','content','label']]])
df.reset_index(drop=True,inplace=True)

Skipping line 836: unexpected end of data


In [ ]:
df['text'] = df['headline'] + df['content']

In [ ]:
df = df.drop(['headline','content'],axis=1)

In [ ]:
df.head()

,label,text
0,1,"হট্টগোল করায় বাকৃবিতে দুইজন বরখাস্ত, ৬ জনকে শোকজগত ১৭ সেপ্টেম্বর বাংলাদেশ কৃষি বিশ্ববিদ্যালয়ে (বাকৃবি) উপাচার্যের কার্যালয়ে হট্টগোলের ঘটনায় দুইজনকে সাময়িক বরখাস্ত ও ছয় জনকে শোকজ করেছে বিশ্ববিদ্যালয় প্রশাসন। বুধবার বিশ্ববিদ্যালয় বাকৃবি রেজিস্ট্রার সাইফুল ইসলাম স্বাক্ষরিত এক নোটিশে আগামী ৭ দিনের মধ্যে উপযুক্ত উত্তর দেয়ার নির্দেশ দেয়া হয়েছে। এদিকে এ ঘটনায় আন্দোলনের সঙ্গে একাত্বতা প্রকাশ না করায় হামলার শিকার হয়ে কারিগরি কর্মচারী পরিষদের সভাপতি ও সাধারণ সম্পাদক হাসপাতালে ভর্তি হয়েছেন। সাময়িক বরখাস্তরা হলেন- শিক্ষা বিষয়ক শাখার কর্মচারী ও ৩য় শ্রেণির সাধারণ সম্পাদক মো. মোশারফ হোসেন ও কর্মকর্তা পরি..."
1,1,মালয়েশিয়ায় কর্মী পাঠানোর ব্যবস্থা নেয়ার সুপারিশবাংলাদেশের বৃহৎ শ্রমবাজার মালয়েশিয়ায় আবার শ্রমিক পাঠানোর প্রয়োজনীয় সব ধরনের ব্যবস্থা নেয়ার সুপারিশ করেছে সংসদীয় কমিটি। ২৫ সেপ্টেম্বর প্রবাসী কল্যাণ ও বৈদেশিক কর্মসংস্থান মন্ত্রণালয়ের মন্ত্রীর নেতৃত্বে মালয়েশিয়ায় জয়েন্ট ওয়ার্কিং গ্রুপের একটি সভা হবে। সভায় শ্রমবাজারে কর্মী পাঠানোর পরিবর্তিত প্রক্রিয়া চূড়ান্তকরণ এবং মালয়েশিয়ার মানবসম্পদ মন্ত্রণালয়সহ অন্যান্য সংশ্লিষ্ট মন্ত্রণালয়ের সঙ্গে নিবিড় যোগাযোগ অব্যাহত রাখার সুপারিশ করবে কমিটি। বুধবার সংসদ ভবনে অনুষ্ঠিত প্রবাসী কল্যাণ ও বৈদেশিক কর্মসংস্থান মন্ত্রণালয় সম্পর্কিত স্থায়ী কমিটির ২৫তম বৈঠকে এ সুপ...
2,1,"প্রেমের প্রস্তাবে রাজি না হওয়ায় স্কুলছাত্রীকে কুপিয়ে জখমনরসিংদীর মনোহরদীতে প্রেমের প্রস্তাবে রাজি না হওয়ায় তানজিমা (১৪) নামে এক স্কুলছাত্রীকে কুপিয়ে জখম করেছে মফিজুল ইসলাম (১৮) নামে এক যুবক। বুধবার সকালে উপজেলার সরাইকান্দী গ্রামে এ ঘটনা ঘটে। গুরুতর আহত অবস্থায় ওই স্কুলছাত্রীকে ঢাকা মেডিকেল কলেজ হাসপাতালে ভর্তি করা হয়েছে। আহত তানজিমা মনোহরদী উপজেলার নলুয়া গ্রামের তারা মিয়ার মেয়ে এবং অর্জুনচর উচ্চ বিদ্যালয়ের নবম শ্রেণির ছাত্রী। স্কুলছাত্রীর স্বজনদের অভিযোগ, দীর্ঘদিন ধরে তানজিমাকে প্রেমের প্রস্তাব দিয়ে আসছিল সরাইকান্দী গ্রামের আব্দুর রশিদ মিয়ার ছেলে মফিজুল ইসলাম। তবে তার প্রস্তাবে রাজি না হওয়..."
3,1,"মেডিয়েশনই মামলাজট নিরসনের পথ : বিচারপতি আহমেদ সোহেলসুপ্রিম কোর্টের হাইকোর্ট বিভাগের বিচারপতি আহমেদ সোহেল বলেছেন, মেডিয়েশন এবং আরবিট্রেশনই মামলাজট নিরসনের একমাত্র পথ। বুধবার (১৯ সেপ্টেম্বর) রাজধানির এক হোটেলে ‘ডেফিনেশন অব মেডিয়েশন’ বইয়ের মোড়ক উন্মোচন অনুষ্ঠানে তিনি এ কথা বলেন। বাংলাদেশ ইন্টারন্যাশনাল মেডিয়েশন সোসাইটির (বিমস) প্রতিষ্ঠাতা অ্যাডভোকেট সমরেন্দ্র নাথ গোস্বামী ও ভারতের সুপ্রিম কোর্টের আইনজীবী অ্যাক্রিডিটেড মেডিয়েটর ইরাম মজিদ যৌথভাবে বইটি লিখেছেন। এর আগে বিমসের দুই দিনব্যাপী আন্তর্জাতিকমানের বিশেষ প্রশিক্ষণ কর্মশালারও উদ্ধোধন করেন বিচারপতি আহমেদ সোহেল। প্রধান অতিথির বক্তেব্য তিনি ব..."
4,1,"টকশোতে বক্তব্য দিতে গিয়ে জাপা নেতার মৃত্যুমাদারীপুর সদরের উপজেলার লেকেরপাড়ে একটি বেসরকারি টেলিভিশন চ্যানেল নিউজ টুয়েন্টিফোরের নির্বাচন-সংক্রান্ত টকশো চলাকালে বক্তব্য দিতে গিয়ে অসুস্থ হয়ে মারা গেছেন জাতীয় পার্টির (জাপা) মাদারীপুর জেলা শাখার সভাপতি জাকারিয়া অপু। মঙ্গলবার তাকে প্রথমে মাদারীপুরের স্থানীয় নিরাময় হাসপাতাল ও পরে ফরিদপুর মেডিকেল কলেজ হাসপাতালে নেয়া হয়। সেখান থেকে ঢাকায় নিয়ে যাওয়ার পথে মারা যান তিনি। জাকারিয়া অপুর পরিবার জানিয়েছে, একটি টেলিভিশনের অনুষ্ঠান চলাকালে হৃদরোগে আক্রান্ত হন জাকারিয়া। পরে হাসপাতালে নেয়ার পথে মারা যান তিনি। মাদারীপুর জেলা জাতীয় পার্টির (জাপা) সহসভাপতি জহিরুল..."


In [ ]:
df.label=df.label.map({1:"Authentic",0:"Fake"})

In [ ]:
#df = df.sample(frac=1,replace=False)

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2)

In [ ]:
train['is_valid'] = False
valid['is_valid'] = True


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train['label'].value_counts()

Fake         1042
Authentic     664
Name: label, dtype: int64

In [ ]:
valid['label'].value_counts()

Fake         257
Authentic    170
Name: label, dtype: int64

In [ ]:
final_df = pd.concat([train,valid],axis=0)

In [ ]:
final_df.head()

,label,text,is_valid
6,Authentic,"হস্তান্তরিত বস্ত্রকলে সরকারের সহায়তার দাবিশ্রমিক-কর্মচারীদের ব্যবস্থাপনায় হস্তান্তরিত বস্ত্রকলগুলো সুষ্ঠু পরিচালনার জন্য সরকারের সহায়তা চেয়েছেন এই মিলগুলোর সমন্বয় পরিষদ। বুধবার শিল্প মন্ত্রণালয়ে শিল্পমন্ত্রী আমির হোসেন আমুর সঙ্গে বৈঠকের সময় এ সহায়তা চান হস্তান্তরিত ৯টি বস্ত্র মিলস সমন্বয় পরিষদের নেতারা। এগুলো সুষ্ঠু পরিচালনার স্বার্থে ২০১৪ সালের অক্টোবরে বস্ত্র ও পাট মন্ত্রণালয় পরিদর্শনকালে প্রধানমন্ত্রীর দেয়া নির্দেশনা দ্রুত বাস্তবায়নের দাবি জানিয়েছেন তারা। বৈঠকে হস্তান্তরিত বস্ত্রকলগুলোর বর্তমান সমস্যা এবং এর সমাধানের উপায় নিয়ে আলোচনা হয়। এ সময় পরিষদের নেতারা বলেন, উন্নত ব্যবস্থাপনার জন্...",False
1296,Fake,"রবীন্দ্রনাথ ও নজরুল দুজনই বিএনপির পক্ষে : কৌতুক যেভাবে সত্যি হলোসাহিত্যিকদের লেখা নানান লাইন নিয়ে রসিকতা, কৌতুক তো হয়ে আসছে সেই কবে থেকেই। বিশেষ করে রবীন্দ্রনাথ ও নজরুলের লেখা বিভিন্ন ক্লাসিক লাইন নিয়ে হয়েছে অনেক eআরকি। সেসব তো আর সিরিয়াসলি নেয়ার কিছু নেই। তবে বিএনপির রুহুল কবির রিজভী সম্ভবত এই রসিকতা একটি সিরিয়াসলিই নিয়ে নিয়েছেন। সাংবাদিকদের উদ্দেশে তিনি বলেন, 'রবীন্দ্রনাথ ও নজরুল দুজনই ছিলেন বিএনপির পক্ষে। কারণ রবীন্দ্রনাথ লিখেছিলেন ""নৌকাডুবি"", আর নজরুল লিখেছেন, ""আমরা শক্তি আমরা বল আমরা ছাত্রদল!""উল্লেখ্য, রবীন্দ্রনাথ এবং নজরুলের এই দুই সাহিত্যকর্ম নিয়ে ঠিক একই রসিকতা eআরকি ডটকমে প্রকাশিত...",False
1856,Fake,উঠতি বয়সী শিক্ষার্থীরা রাত ৮টার পরে বাইরে থাকলে আটক করা হবে!গতকাল রাজবাড়ীতে এক সচেতনতামূলক অনুষ্ঠানে রাজবাড়ীর পুলিশ সুপার (এসপি) উঠতি বয়সী কোনো শিক্ষার্থী রাত আটটার পর বাসার বাইরে থাকলে তাকে আটক করে থানায় নিয়ে যাওয়ার নির্দেশ দিয়েছেন। চলমান সামাজিক অবক্ষয় রোধের পরিপ্রেক্ষিতে তিনি এ নির্দেশ দেন।দেশে যুবক সমাজের মধ্যে মাদকদ্রব্য সেবন করার প্রবণতা অনেক বৃদ্ধি পেয়েছে। যার ফলে দেশ ও জাতি আজ হুমকীর মুখে। এ প্রবণতা নির্মুল করার জন্য দেশের বিভিন্ন যায়গায় সভার আয়োজন করা হচ্ছে।গতকাল দুপুরে রাজবাড়ী শহরের ডা. আবুল হোসেন কলেজ কর্তৃপক্ষ শিক্ষক-শিক্ষার্থী ও অভিভাবকদের নিয়ে এক সচেতনতামূলক অনুষ্ঠানের আয়োজন...,False
1061,Fake,"গবেষণায় দাবি- বউ হিসাবে খাটো মেয়েরা ভালোবিয়ে মানুষের জীবনে বিয়ে সব’চে গুরুত্বপূর্ণ বিষয়। বেশিরভাগ মানুষ বিয়ের জন্য লম্বা মেয়ে খুজে থাকে। ভালো সঙ্গি হিসাবে মেয়েটির উচ্চতাকেই প্রাধান্য দিয়ে থাকে। তবে সম্প্রতি এক জরিপে প্রকাশ হয়েছে। বউ হিসাবে খাটো মেয়েরা ভালো। বউ যদি স্বামীর চেয়ে খাটো হয়, তাইলে সে বউ বেশি স্বামী ভক্ত হয়ে থাকে।এ ছাড়া বউ যদি খাটো হয়, তাইলে নানারকম সুবিধা পাওয়া যায়-বউ খাটো হলে কী কী সুবিধা পাবেন তা জানিয়েছে জীবনধারা বিষয়ক ওয়েবসাইট বোল্ডস্কাই।১. জড়িয়ে ধরার ‘অসাধারণ’ অনুভূতিঃ-ভালবাসার মানুষকে জড়িয়ে ধরে সুখ কে না পেতে চাই? সেক্ষেত্রে বউ যদি খাটো হয়, তাইলে তাকে বুকের মাঝে জড়িয়ে ধরা ...",False
700,Authentic,"রাজশাহীতে যুবককে পিটিয়ে হত্যার অভিযোগরাজশাহী: রাজশাহীর রামচন্দ্রপুরে (মিরেরচক) চাচাতো ভাইদের মারধরে লিটন নামের এক যুবক নিহত হয়েছেন বলে অভিযোগ উঠেছে। এ ঘটনায় পুলিশ তিনজনকে আটক করেছে। বুধবার (১৯ সেপ্টেম্বর) রাত ১টার দিকে রাজশাহী মেডিকেল কলেজ (রামেক) হাসপাতালে চিকিৎসাধীন অবস্থায় তার মৃত্যু হয়। মঙ্গলবার (১৮ সেপ্টেম্বর) সন্ধ্যায় মারধরের ঘটনা ঘটে। নিহত লিটনের বড় ভাই আলাউদ্দীন জানান, বাড়ির রাস্তায় চাচাতো ভাইয়েরা ময়লা-আবর্জনা ফেলতো। মঙ্গলবার সন্ধ্যায় বাড়ি যাওয়ার সময় রাস্তায় ময়লা ফেলতে নিষেধ করেন লিটন। এ সময় তার চাচাতো ভাই মঞ্জুর ও মাহমুদের সঙ্গে বাকবিতণ্ডা হয়। কথা কাটাকাটির একপর্যায়ে তারা লিটন...",False


# Model

In [ ]:
wandb.init(**wandb_init_kwargs)

wandb: Currently logged in as: tasmiah-tahsin (use `wandb login --relogin` to force relogin)


# Explaining predictions

# Model Sharing